In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [20]:
llm = ChatOpenAI()

In [21]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [22]:
embeddings = OpenAIEmbeddings()

In [23]:
from pathlib import Path
p = Path('../data')

In [24]:
data_files = list(p.glob('**/*.txt'))

In [25]:
documents = []

In [26]:
for file in data_files:
    with open(file) as f:
        content = f.readlines()
        documents.append(content)
        

In [27]:
len(documents), len(data_files)

(4, 4)

In [29]:
# vector = FAISS.load_local()

In [30]:
vector = FAISS.from_documents(documents, embeddings)

AttributeError: 'list' object has no attribute 'page_content'

In [7]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

In [8]:
document_chain = create_stuff_documents_chain(llm, prompt)


retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare different configurations side-by-side, provide a playground environment for rapid iteration and experimentation, enable beta testing to collect real-world performance data, capture feedback from users, annotate traces for evaluation criteria, add runs to datasets for refining and improving performance, monitor key metrics over time, perform A/B testing, and use automations to process traces in near real-time.
